
# Estimation d'âge à partir d'images — PyTorch (Régression **ou** Classification douce)

Ce notebook permet de **choisir** entre deux approches :
- **Régression** (MAE/Huber)
- **Classification douce** avec **soft labels** (distribution 0..100 ans + espérance)

Il inclut :
- parsing des noms `XXXXXX_YZWW.ext`,
- split **par personne (ID)** pour éviter la fuite d'info,
- `AgeDataset` + DataLoaders,
- deux modèles : `AgeRegressor` et `AgeClassifier`,
- boucles d'entraînement/évaluation adaptées,
- inférence test + **export CSV** + sauvegarde des poids.

> ⚙️ Change le bloc **Configuration** pour sélectionner la méthode.


In [ ]:

# =========================
# Configuration générale
# =========================
from pathlib import Path

# Dossiers à adapter
TRAIN_DIR = Path("data/train")   # chemin vers les images d'entraînement
TEST_DIR  = Path("data/test")    # chemin vers les images de test

# Choix de la méthode: 'regression' ou 'classification'
METHOD = "classification"  # <-- change ici

# Paramètres communs
BACKBONE = "convnext_tiny"       # ex: "resnet50", "efficientnetv2_s", "convnext_tiny"
USE_SEX  = True                  # utilise la variable sexe (1=M, 0=F) concaténée aux features
MAX_AGE  = 100                   # bornes d'âge (0..MAX_AGE)
IMG_SIZE = 224                   # taille d'entrée

BATCH_SIZE = 64
EPOCHS     = 20
LR         = 3e-4
WD         = 1e-4
SEED       = 42

# Pour classification douce
HIDDEN_DIM = 512
SIGMA_SOFT = 2.0                 # écart-type pour soft labels gaussiens

# Fichier de sortie
EXPERIMENT_NAME = f"age_{METHOD}_{BACKBONE}_{datetime.now().strftime('%Y%m%d_%H%M')}"
BEST_WEIGHTS = f"{EXPERIMENT_NAME}_best.pt"
SUBMISSION_CSV = f"{EXPERIMENT_NAME}_submission.csv"

print("Méthode:", METHOD)
print("Backbone:", BACKBONE)
print("Train dir:", TRAIN_DIR.resolve())
print("Test dir:", TEST_DIR.resolve())


In [ ]:

# =========================
# Imports & seed
# =========================
import os, re, math, random
import numpy as np
import pandas as pd
from PIL import Image

import torch
import torch.nn as nn
import torch.backends.cudnn as cudnn
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
import timm

from sklearn.model_selection import StratifiedGroupKFold
from sklearn.metrics import precision_score, recall_score, f1_score

def set_seed(seed=42):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    cudnn.deterministic = True
    cudnn.benchmark = False

set_seed(SEED)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Device:", device)


In [ ]:

# =========================
# Parsing des noms & Dataset
# =========================
IMG_EXTS = {".jpg", ".jpeg", ".png", ".bmp", ".webp", ".JPG", ".JPEG", ".PNG"}
_AGE_RE = re.compile(r'^(?P<pid>\d+)_(?P<idx>\d{1,2})(?P<sex>[MF])(?P<age>\d{2})$')

def parse_name(fname: str):
    stem = Path(fname).stem
    m = _AGE_RE.match(stem)
    if not m:
        raise ValueError(f"Nom non conforme: {fname}")
    d = m.groupdict()
    return int(d["pid"]), int(d["idx"]), d["sex"], int(d["age"])

def age_bucket(age, width=5):
    return age // width

class AgeDataset(Dataset):
    def __init__(self, img_dir, items, transform=None):
        self.img_dir = Path(img_dir)
        self.items = items  # list of dicts: {"path": str, "age": int, "sex": 0/1}
        self.transform = transform

    def __len__(self):
        return len(self.items)

    def __getitem__(self, i):
        it = self.items[i]
        path = self.img_dir / it["path"]
        img = Image.open(path).convert("RGB")
        if self.transform:
            img = self.transform(img)
        age = torch.tensor([it["age"]], dtype=torch.float32)
        sex = torch.tensor([it["sex"]], dtype=torch.float32)
        return img, age, sex, it["path"]


In [ ]:

# =========================
# Construction DF & split par personne (ID)
# =========================
rows = []
for p in sorted(TRAIN_DIR.iterdir()):
    if p.suffix in IMG_EXTS:
        pid, idx, sex, age = parse_name(p.name)
        rows.append({"filename": p.name, "person_id": pid, "photo_idx": idx,
                     "sex": sex, "age": age})
df = pd.DataFrame(rows)
df["age_bucket"] = df["age"].apply(age_bucket)
print("Images train:", len(df), "| personnes uniques:", df.person_id.nunique())
display(df.head()) if len(df) else None

# StratifiedGroupKFold: stratifie par age_bucket tout en groupant par person_id
sgkf = StratifiedGroupKFold(n_splits=5, shuffle=True, random_state=SEED)
X = np.zeros(len(df))
y = df["age_bucket"].values
groups = df["person_id"].values

train_idx, val_idx = next(iter(sgkf.split(X, y, groups)))

train_df = df.iloc[train_idx].copy()
val_df   = df.iloc[val_idx].copy()

assert set(train_df.person_id) & set(val_df.person_id) == set(), "Fuite détectée: IDs communs."

print("Split -> train images:", len(train_df), "val images:", len(val_df))

# Build items for Dataset
def to_items(subdf):
    items = []
    for _, r in subdf.iterrows():
        items.append({
            "path": r["filename"],
            "age": int(r["age"]),
            "sex": 1 if r["sex"] == "M" else 0
        })
    return items

train_items = to_items(train_df)
val_items   = to_items(val_df)

# Test items
test_items = [{"path": p.name} for p in sorted(TEST_DIR.iterdir()) if p.suffix in IMG_EXTS]

print("Aperçu item:", train_items[0] if train_items else None)


In [ ]:

# =========================
# Transforms
# =========================
mean, std = [0.485,0.456,0.406], [0.229,0.224,0.225]

train_tf = transforms.Compose([
    transforms.Resize(256),
    transforms.RandomResizedCrop(IMG_SIZE, scale=(0.8,1.0)),
    transforms.RandomHorizontalFlip(),
    transforms.ColorJitter(0.1,0.1,0.1,0.05),
    transforms.ToTensor(),
    transforms.Normalize(mean, std),
    transforms.RandomErasing(p=0.25),
])

val_tf = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(IMG_SIZE),
    transforms.ToTensor(),
    transforms.Normalize(mean, std),
])

class TestDataset(Dataset):
    def __init__(self, img_dir, items, transform=None):
        self.img_dir = Path(img_dir); self.items = items; self.transform = transform
    def __len__(self): return len(self.items)
    def __getitem__(self, i):
        p = self.items[i]["path"]
        img = Image.open(self.img_dir / p).convert("RGB")
        if self.transform: img = self.transform(img)
        sex = torch.tensor([0.5], dtype=torch.float32)  # neutre en test si inconnu
        return img, sex, p


In [ ]:

# =========================
# DataLoaders
# =========================
num_workers = 4

train_ds = AgeDataset(TRAIN_DIR, train_items, transform=train_tf)
val_ds   = AgeDataset(TRAIN_DIR, val_items,   transform=val_tf)
test_ds  = TestDataset(TEST_DIR, test_items,  transform=val_tf)

train_loader = DataLoader(train_ds, batch_size=BATCH_SIZE, shuffle=True,  num_workers=num_workers, pin_memory=True)
val_loader   = DataLoader(val_ds,   batch_size=BATCH_SIZE, shuffle=False, num_workers=num_workers, pin_memory=True)
test_loader  = DataLoader(test_ds,  batch_size=BATCH_SIZE, shuffle=False, num_workers=num_workers, pin_memory=True)

len(train_loader), len(val_loader), len(test_loader)


In [ ]:

# =========================
# Modèles
# =========================
class AgeRegressor(nn.Module):
    def __init__(self, backbone_name=BACKBONE, use_sex=USE_SEX):
        super().__init__()
        self.backbone = timm.create_model(backbone_name, pretrained=True, num_classes=0, global_pool="avg")
        feat_dim = self.backbone.num_features
        self.use_sex = use_sex
        in_dim = feat_dim + (1 if use_sex else 0)
        self.head = nn.Sequential(
            nn.Dropout(0.2),
            nn.Linear(in_dim, 1)
        )
    def forward(self, x, sex=None):
        f = self.backbone(x)
        if self.use_sex and sex is not None:
            f = torch.cat([f, sex], dim=1)
        out = self.head(f)  # (B,1)
        return out

class AgeClassifier(nn.Module):
    def __init__(self, backbone_name=BACKBONE, max_age=MAX_AGE, use_sex=USE_SEX, hidden_dim=HIDDEN_DIM):
        super().__init__()
        self.backbone = timm.create_model(backbone_name, pretrained=True, num_classes=0, global_pool="avg")
        feat_dim = self.backbone.num_features
        self.use_sex = use_sex
        in_dim = feat_dim + (1 if use_sex else 0)
        self.head = nn.Sequential(
            nn.Linear(in_dim, hidden_dim),
            nn.ReLU(inplace=True),
            nn.Dropout(0.30),
            nn.Linear(hidden_dim, max_age + 1)  # logits 0..max_age
        )
    def forward(self, x, sex=None):
        f = self.backbone(x)
        if self.use_sex and sex is not None:
            f = torch.cat([f, sex], dim=1)
        logits = self.head(f)             # (B, max_age+1)
        probs  = torch.softmax(logits, 1)
        return logits, probs


In [ ]:

# =========================
# Pertes & utilitaires
# =========================
def soft_labels(ages_float, max_age=MAX_AGE, sigma=SIGMA_SOFT, device=None):
    device = device or ages_float.device
    ages_grid = torch.arange(0, max_age+1, device=device).float()  # [0..max_age]
    d2 = (ages_grid.unsqueeze(0) - ages_float.unsqueeze(1))**2
    dist = torch.exp(-0.5 * d2 / (sigma**2))
    dist = dist / (dist.sum(1, keepdim=True) + 1e-8)
    return dist

def classification_loss(logits, target_soft):
    return nn.KLDivLoss(reduction="batchmean")(torch.log_softmax(logits, dim=1), target_soft)

# métriques
def compute_metrics(y_true, y_pred):
    import numpy as np, math
    y_true = np.array(y_true, dtype=float)
    y_pred = np.array(y_pred, dtype=float)
    mae  = np.mean(np.abs(y_pred - y_true))
    rmse = math.sqrt(np.mean((y_pred - y_true)**2))
    def within_k(k): return 100.0 * np.mean(np.abs(y_pred - y_true) <= k)
    # “classification tolérante” ±2 ans
    y_ok_true = np.ones_like(y_true)
    y_ok_pred = (np.abs(y_pred - y_true) <= 2).astype(int)
    P = precision_score(y_ok_true, y_ok_pred, zero_division=0)
    R = recall_score(y_ok_true, y_ok_pred, zero_division=0)
    F1 = f1_score(y_ok_true, y_ok_pred, zero_division=0)
    return {
        "MAE": mae, "RMSE": rmse,
        "Within_1(%)": within_k(1),
        "Within_2(%)": within_k(2),
        "Within_3(%)": within_k(3),
        "Prec_tol±2": P, "Rec_tol±2": R, "F1_tol±2": F1
    }


In [ ]:

# =========================
# Boucles d'entraînement & évaluation
# =========================
def train_one_epoch_reg(model, loader, optimizer, scaler, device, criterion, max_grad_norm=1.0):
    model.train()
    loss_sum, n = 0.0, 0
    for imgs, ages, sex, _ in loader:
        imgs = imgs.to(device, non_blocking=True)
        ages = ages.to(device, non_blocking=True)
        sex  = sex.to(device, non_blocking=True)
        optimizer.zero_grad(set_to_none=True)
        with torch.cuda.amp.autocast(True):
            preds = model(imgs, sex).clamp(0, float(MAX_AGE))
            loss = criterion(preds, ages)
        scaler.scale(loss).backward()
        scaler.unscale_(optimizer)
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm)
        scaler.step(optimizer)
        scaler.update()
        bs = imgs.size(0); loss_sum += loss.item() * bs; n += bs
    return loss_sum / max(1,n)

@torch.no_grad()
def evaluate_reg(model, loader, device):
    model.eval()
    y_true, y_pred = [], []
    for imgs, ages, sex, _ in loader:
        imgs = imgs.to(device, non_blocking=True)
        ages = ages.to(device, non_blocking=True)
        sex  = sex.to(device, non_blocking=True)
        preds = model(imgs, sex).clamp(0, float(MAX_AGE)).squeeze(1)
        y_true.extend(ages.squeeze(1).cpu().numpy().tolist())
        y_pred.extend(preds.cpu().numpy().tolist())
    return compute_metrics(y_true, y_pred)

def train_one_epoch_cls(model, loader, optimizer, scaler, device, max_grad_norm=1.0):
    model.train()
    loss_sum, n = 0.0, 0
    for imgs, ages, sex, _ in loader:
        imgs = imgs.to(device, non_blocking=True)
        sex  = sex.to(device, non_blocking=True)
        ages = ages.squeeze(1).to(device, non_blocking=True)
        optimizer.zero_grad(set_to_none=True)
        with torch.cuda.amp.autocast(True):
            logits, probs = model(imgs, sex)
            target = soft_labels(ages, max_age=MAX_AGE, sigma=SIGMA_SOFT)
            loss = classification_loss(logits, target)
        scaler.scale(loss).backward()
        scaler.unscale_(optimizer)
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm)
        scaler.step(optimizer)
        scaler.update()
        bs = imgs.size(0); loss_sum += loss.item() * bs; n += bs
    return loss_sum / max(1,n)

@torch.no_grad()
def evaluate_cls(model, loader, device):
    model.eval()
    y_true, y_pred = [], []
    ages_grid = torch.arange(0, MAX_AGE+1, device=device).float()
    for imgs, ages, sex, _ in loader:
        imgs = imgs.to(device, non_blocking=True)
        sex  = sex.to(device, non_blocking=True)
        ages = ages.squeeze(1).to(device, non_blocking=True)
        logits, probs = model(imgs, sex)
        pred = torch.sum(probs * ages_grid.unsqueeze(0), dim=1)  # espérance
        y_true.extend(ages.cpu().numpy().tolist())
        y_pred.extend(pred.cpu().numpy().tolist())
    return compute_metrics(y_true, y_pred)


In [ ]:

# =========================
# Entraînement principal
# =========================
scaler = torch.amp.GradScaler()
best_mae, best_path = 1e9, BEST_WEIGHTS

if METHOD == "regression":
    model = AgeRegressor(BACKBONE, USE_SEX).to(device)
    criterion = nn.SmoothL1Loss(beta=1.0)  # Huber
    optimizer = torch.optim.AdamW(model.parameters(), lr=LR, weight_decay=WD)

    for epoch in range(1, EPOCHS+1):
        tr_loss = train_one_epoch_reg(model, train_loader, optimizer, scaler, device, criterion)
        metrics = evaluate_reg(model, val_loader, device)
        print(f"[{epoch:02d}] train_loss={tr_loss:.4f} | MAE={metrics['MAE']:.3f} | W2={metrics['Within_2(%)']:.1f}% | F1tol2={metrics['F1_tol±2']:.3f}")
        if metrics["MAE"] < best_mae:
            best_mae = metrics["MAE"]
            torch.save(model.state_dict(), best_path)

elif METHOD == "classification":
    model = AgeClassifier(BACKBONE, MAX_AGE, USE_SEX, HIDDEN_DIM).to(device)
    optimizer = torch.optim.AdamW(model.parameters(), lr=LR, weight_decay=WD)

    for epoch in range(1, EPOCHS+1):
        tr_loss = train_one_epoch_cls(model, train_loader, optimizer, scaler, device)
        metrics = evaluate_cls(model, val_loader, device)
        print(f"[{epoch:02d}] train_loss={tr_loss:.4f} | MAE={metrics['MAE']:.3f} | W2={metrics['Within_2(%)']:.1f}% | F1tol2={metrics['F1_tol±2']:.3f}")
        if metrics["MAE"] < best_mae:
            best_mae = metrics["MAE"]
            torch.save(model.state_dict(), best_path)

print("Best MAE:", best_mae, "| saved:", best_path)


In [ ]:

# =========================
# Inférence sur test + export CSV
# =========================
# Recharge le meilleur modèle
if METHOD == "regression":
    model = AgeRegressor(BACKBONE, USE_SEX).to(device)
    model.load_state_dict(torch.load(BEST_WEIGHTS, map_location=device))
    model.eval()

    preds, names = [], []
    with torch.no_grad():
        for imgs, sex, paths in test_loader:
            imgs = imgs.to(device)
            sex  = sex.to(device)
            y = model(imgs, sex).clamp(0, float(MAX_AGE)).squeeze(1)
            preds.extend(torch.round(y).cpu().numpy().tolist())  # arrondi à l'entier pour soumission
            names.extend(paths)

elif METHOD == "classification":
    model = AgeClassifier(BACKBONE, MAX_AGE, USE_SEX, HIDDEN_DIM).to(device)
    model.load_state_dict(torch.load(BEST_WEIGHTS, map_location=device))
    model.eval()

    preds, names = [], []
    ages_grid = torch.arange(0, MAX_AGE+1, device=device).float()
    with torch.no_grad():
        for imgs, sex, paths in test_loader:
            imgs = imgs.to(device)
            sex  = sex.to(device)
            logits, probs = model(imgs, sex)
            y = torch.sum(probs * ages_grid.unsqueeze(0), dim=1)
            y = torch.round(y).clamp(0, float(MAX_AGE))  # arrondi pour soumission
            preds.extend(y.cpu().numpy().tolist())
            names.extend(paths)

sub_df = pd.DataFrame({"filename": names, "age": [int(x) for x in preds]})
sub_df.to_csv(SUBMISSION_CSV, index=False)
print("Submission saved:", SUBMISSION_CSV)
sub_df.head()
